In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import numpy as np
import itertools
import logging
from pathlib import Path
import hashlib

In [ ]:
# healthy_emory = pd.read_csv('emory_final.csv')
healthy_vini = pd.read_csv('vinidir_final.csv')

In [ ]:
# healthy_emory = emory[emory.category.isnull()].reset_index(drop=True)
# healthy_vini = vini[vini.category=='No Finding'].reset_index(drop=True)

In [ ]:
# healthy_vini = healthy_vini.rename(columns={'image_paths':'image_path'})
# healthy_emory = healthy_emory.rename(columns={'Image_Paths':'image_path'})

In [ ]:
# healthy_emory['image_path'] = healthy_emory['image_path'].str.replace(r'^../', '', regex=True)

In [ ]:
# healthy_emory = healthy_emory[(healthy_emory['asses']!='BIRADS 0') & (healthy_emory['asses']!='BIRADS 6')].reset_index(drop=True)

# healthy_emory['asses'] = healthy_emory['asses'].str.replace(' ', '').str.upper()
# # vini['breast_birads'] = vini['breast_birads'].str.replace('-', '').str.replace(' ', '').str.upper()

# # Rename columns for clarity and merge datasets
# healthy_emory = healthy_emory.rename(columns={'asses': 'breast_birads', 'ViewPosition':'view','ImageLateralityFinal':'laterality'})

In [ ]:
# healthy_emory.breast_birads.value_counts()
healthy_vini.breast_birads.value_counts()

In [ ]:
# em1 = emory_f[emory_f['breast_birads']=='BIRADS1'].reset_index(drop=True)
# vn1 = healthy_vini[healthy_vini['breast_birads']=='BI-RADS 1'].reset_index(drop=True)

In [ ]:
# # # Filter out rows with BIRADS5 in either breast_birads_CC or breast_birads_MLO
# # healthy_emory = healthy_emory[~healthy_emory['breast_birads'].isin(['BIRADS5'])]
# # Get the count for each level in the combined dataset
# level_counts = healthy_emory['breast_birads'].value_counts()

# # Determine the minimum count across levels for balanced sampling
# min_count = level_counts.min()

# # Sample the minimum number of records for each BIRADS level
# healthy_emory = healthy_emory.groupby('breast_birads', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42))

# # Verify the balanced distribution
# healthy_emory['breast_birads'].value_counts()

In [ ]:
# # # Filter out rows with BIRADS5 in either breast_birads_CC or breast_birads_MLO
# # healthy_emory = healthy_emory[~healthy_emory['breast_birads'].isin(['BIRADS5'])]
# # Get the count for each level in the combined dataset
# level_counts = healthy_vini['breast_birads'].value_counts()

# # Determine the minimum count across levels for balanced sampling
# min_count = level_counts.min()

# # Sample the minimum number of records for each BIRADS level
# healthy_vini = healthy_vini.groupby('breast_birads', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42))

# # Verify the balanced distribution
# healthy_vini['breast_birads'].value_counts()

In [ ]:
def display_grid(data):
    idx = random.sample(range(data.shape[0]), 4)
    
    # Set the plot size
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(30, 10))
    axes = axes.ravel()

    for i, row in enumerate(data.iloc[idx].itertuples()):
        # Build image path
        img_file_path = row.image_paths

        # Open and show image
        if os.path.exists(img_file_path):
            img = Image.open(img_file_path)
            try:
                axes[i].imshow(img, cmap='gray')
            except Exception as e:
                print(f"Cannot Display {img_file_path}, because of error:{e}")
        else:
            axes[i].text(0.5, 0.5, 'Image not found', horizontalalignment='center', verticalalignment='center')
        
        # Set title with image attributes
        title_text = f"{i} Prompt {row.prompt}"
        axes[i].set_title(title_text, fontsize=8)
        # axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()



In [ ]:
# em1['breast_density_prompt'] = em1['tissueden'].apply(lambda x: 'very low' if x == 'DENSITY A' else 'low' if x == 'DENSITY B' else 'high' if x == 'DENSITY C' else 'very high')
# em1['prompt'] = 'a hologic mammogram in ' + em1['view'] + ' view ' + 'with ' + em1['breast_density_prompt'] + ' density with breast BIRADS 1 Level'

# vn1['breast_density_prompt'] = vn1['breast_density'].apply(lambda x: 'very low' if x == 'DENSITY A' else 'low' if x == 'DENSITY B' else 'high' if x == 'DENSITY C' else 'very high')
# vn1['prompt'] = 'a siemens mammogram in ' + vn1['view'] + ' view ' + 'with ' + vn1['breast_density_prompt'] + ' density with breast BIRADS 1 Level'

# healthy_emory['prompt'] = "A mammogram"
healthy_vini["prompt"] = "A mammogram"

In [ ]:
# display_grid(healthy_emory.iloc[:4])

In [ ]:
display_grid(healthy_vini)

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

def torch_CountUpContinuingOnes(b_arr):
    left = torch.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = torch.cummax(left, dim=-1)[0]

    rev_arr = torch.flip(b_arr, [-1])
    right = torch.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = torch.cummax(right, dim=-1)[0]
    right = len(rev_arr) - 1 - torch.flip(right, [-1])

    return right - left - 1

def torch_ExtractBreast_with_padding_single_side(img_ori, target_size=(512, 512), padding=1):
    # Detect background and set to zero
    img = torch.where(img_ori <= 20, torch.zeros_like(img_ori), img_ori)
    height, _ = img.shape

    # Extract the main breast region (same as before)
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].to(torch.float32).std(dim=0) != 0
    continuing_ones = torch_CountUpContinuingOnes(b_arr)
    col_ind = torch.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]

    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].to(torch.float32).std(dim=1) != 0
    continuing_ones = torch_CountUpContinuingOnes(b_arr)
    row_ind = torch.where(continuing_ones == continuing_ones.max())[0]
    breast_region = img_ori[row_ind][:, col_ind]

    # Resize the extracted breast region while maintaining the aspect ratio
    breast_height, breast_width = breast_region.shape
    aspect_ratio = breast_width / breast_height

    # Define target dimensions based on aspect ratio
    if aspect_ratio > 1:
        # Wider than tall
        new_width = target_size[1] - padding
        new_height = int(new_width / aspect_ratio)
    else:
        # Taller than wide
        new_height = target_size[0] - padding
        new_width = int(new_height * aspect_ratio)

    resized_breast = cv2.resize(breast_region.cpu().numpy(), (new_width, new_height))
    resized_breast = torch.from_numpy(resized_breast)

    # Determine which side has lower intensity
    pad_x = target_size[1] - new_width
    pad_y = target_size[0] - new_height

    # Initialize offsets
    x_offset = 0
    y_offset = 0

    # Decide padding side for x-axis
    if pad_x > 0:
        left_intensity = resized_breast[:, 0].mean()
        right_intensity = resized_breast[:, -1].mean()
        if left_intensity < right_intensity:
            # Pad on the left side
            x_offset = pad_x
        else:
            # Pad on the right side
            x_offset = 0

    # Decide padding side for y-axis
    if pad_y > 0:
        top_intensity = resized_breast[0, :].mean()
        bottom_intensity = resized_breast[-1, :].mean()
        if top_intensity < bottom_intensity:
            # Pad on the top side
            y_offset = pad_y
        else:
            # Pad on the bottom side
            y_offset = 0

    # Create a padded image with the target size and place the resized breast region
    padded_img = torch.zeros(target_size, dtype=resized_breast.dtype)
    padded_img[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_breast

    return padded_img



# Apply the function in display_grid_transform for Emory dataset

def resize_with_padding_vini(img_np, target_size=(512, 512), padding=1):
    # Convert image to a torch tensor if not already
    img_torch = torch.from_numpy(img_np).to(torch.float32)

    # Ensure the breast region fits within a consistent scale
    breast_height, breast_width = img_torch.shape
    aspect_ratio = breast_width / breast_height

    # Define target dimensions based on aspect ratio
    if aspect_ratio > 1:
        # Wider than tall
        new_width = target_size[1] - padding
        new_height = int(new_width / aspect_ratio)
    else:
        # Taller than wide
        new_height = target_size[0] - padding
        new_width = int(new_height * aspect_ratio)
    
    # Resize while maintaining the aspect ratio
    resized_breast = cv2.resize(img_np, (new_width, new_height))
    resized_breast = torch.from_numpy(resized_breast)

     # Determine which side has lower intensity
    pad_x = target_size[1] - new_width
    pad_y = target_size[0] - new_height

    # Initialize offsets
    x_offset = 0
    y_offset = 0

    # Decide padding side for x-axis
    if pad_x > 0:
        left_intensity = resized_breast[:, 0].mean()
        right_intensity = resized_breast[:, -1].mean()
        if left_intensity < right_intensity:
            # Pad on the left side
            x_offset = pad_x
        else:
            # Pad on the right side
            x_offset = 0

    # Decide padding side for y-axis
    if pad_y > 0:
        top_intensity = resized_breast[0, :].mean()
        bottom_intensity = resized_breast[-1, :].mean()
        if top_intensity < bottom_intensity:
            # Pad on the top side
            y_offset = pad_y
        else:
            # Pad on the bottom side
            y_offset = 0

    # Create a padded image with the target size and place the resized breast region
    padded_img = torch.zeros(target_size, dtype=resized_breast.dtype)
    padded_img[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_breast

    return padded_img


# Apply in display_grid_transform for the Vini dataset



In [ ]:
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_img = clahe.apply(image)
    return equalized_img

def mean_variance_normalization(img_torch):
    img_min, img_max = img_torch.min(), img_torch.max()
    img_torch = (img_torch - img_min) / (img_max - img_min) * 255
    return img_torch



In [1]:
def resize_image(image_path, size=(512, 512)):
    if os.path.exists(image_path):
        img_np = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Check if the image was read successfully
        if img_np is None:
            logging.error(f"Failed to load image at {image_path}.")
            return None  # You could return a default image or raise an error
            
        if 'emory' in image_path:
            img_torch = torch.from_numpy(img_np).to(torch.float32)
            img_torch = mean_variance_normalization(img_torch)

            # Extract the breast region
            breast_region = torch_ExtractBreast_with_padding_single_side(img_torch)

            # Convert the result back to a NumPy array for visualization or further processing
            breast_region_np = breast_region.cpu().numpy().astype(np.uint8)
            # breast_region_np = apply_clahe(breast_region_np)
            breast_region_rgb = np.repeat(breast_region_np[:, :, np.newaxis], 3, axis=2)

            # Convert the NumPy array to a PIL image for compatibility with transforms
            breast_region_pil = Image.fromarray(breast_region_rgb)
        else:
            # img_torch = torch.from_numpy(img_np).to(torch.float32)
            # # Normalize the image
            img_np = mean_variance_normalization(img_np)
            breast_region_np = resize_with_padding_vini(img_np)
            # Convert the result back to a NumPy array for visualization or further processing
            breast_region_np = breast_region_np.cpu().numpy().astype(np.uint8)
            # breast_region_np = apply_clahe(breast_region_np)

            breast_region_rgb = np.repeat(breast_region_np[:, :, np.newaxis], 3, axis=2)

            # Convert the NumPy array to a PIL image for compatibility with transforms
            breast_region_pil = Image.fromarray(breast_region_rgb)

        img = breast_region_pil
        return img
    else:
        logging.error(f"Image path does not exist: {image_path}")
        return None


In [ ]:
from torchvision import transforms

exp_tr = transforms.Compose(
    [
        transforms.Resize(size=(512, 512), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.5],[0.5])
    ]
)
    
def display_grid_transform(data):
    idx = random.sample(range(data.shape[0]), 4)
    
    # Set the plot size
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(30, 10))
    axes = axes.ravel()

    for i, row in enumerate(data.iloc[idx].itertuples()):
        # Build image path
        img_file_path = row.image_paths

        # Open and show image
        if os.path.exists(img_file_path):
            img = resize_image(img_file_path)
            img_t = exp_tr(img)
            try:
                axes[i].imshow(img_t.numpy().transpose((1,2,0)))
            except Exception as e:
                print(f"Cannot Display {img_file_path}, because of error:{e}")
        else:
            axes[i].text(0.5, 0.5, 'Image not found', horizontalalignment='center', verticalalignment='center')
        
        # Set title with image attributes
        title_text = f"{i} Prompt {row.prompt}"
        axes[i].set_title(title_text, fontsize=8)
        # axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()



In [ ]:
# display_grid_transform(healthy_emory.iloc[:4])

In [ ]:
display_grid_transform(healthy_vini)

In [ ]:
# data = pd.concat([
#     healthy_emory[['image_path', 'prompt']],
#     healthy_vini[['image_path', 'prompt']]
# ], axis=0).reset_index(drop=True)
data = healthy_vini[['image_paths', 'prompt']]

from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
# Drop rows where 'prompt' is not a string
train = train[train['prompt'].apply(lambda x: isinstance(x, str))].reset_index(drop=True)

In [ ]:
# instance_prompt = "A mammogram"
# validation_prompt = "A mammogram in CC view with high density"

In [ ]:
# # Find all rows where 'prompt' is not a string
# non_string_prompts = train[~train['prompt'].apply(lambda x: isinstance(x, str))]

# # Display the result
# print(non_string_prompts)


In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms


class DreamBoothDataset(Dataset):
    def __init__(
        self, 
        dataset, 
        tokenizer, 
        class_data_root=None,
        class_prompt=None,
        class_num=None,
        size=512):
        self.dataset = dataset
        # self.instance_prompt = instance_prompt
        self.tokenizer = tokenizer
        self.size = size
        self.transforms = transforms.Compose(
            [
                transforms.Resize(size=(size, size), interpolation=transforms.InterpolationMode.BILINEAR),
                transforms.RandomHorizontalFlip(0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5, 0.5, 0.5]),
            ]
        )
        
        if class_data_root is not None: # if there are prior images
            self.class_data_root = Path(class_data_root)
            self.class_data_root.mkdir(parents=True, exist_ok=True) # create the folder if it doesn't exist
            self.class_images_path = [image_path for image_path in self.class_data_root.glob("*jpg")] # get paths of all the class images
            if class_num is not None: # class number. This can vary if there are more images in the folder and we only want to use a subset of them
                self.num_class_images = min(len(self.class_images_path), class_num)
            else:
                self.num_class_images = len(self.class_images_path)
            self._length = max(self.num_class_images, self.dataset.shape[0]) # length of the dataset will be the max of the number of instance images and the number of class images
            self.class_prompt = class_prompt
        else:
            self.class_data_root = None
        

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        example = {}
        image_path = self.dataset.image_paths[index]
        instance_prompt = self.dataset.prompt[index]

        img = resize_image(image_path, (self.size, self.size))

        # Retry with a different image or handle the None case
        if img is None:
            logging.warning(f"Image loading failed at index {index}. Using a fallback image.")
            image_path = self.dataset.image_path[index-1]
            img = resize_image(image_path, (self.size, self.size))
            
        if img:
            img_t = self.transforms(img)

        example["instance_images"] = img_t

        # Convert tokenizer output to tensor immediately
        example["instance_prompt_ids"] = self.tokenizer(
            instance_prompt,
            padding="max_length",
            truncation=True,
            max_length=self.tokenizer.model_max_length,
            return_tensors="pt"
        ).input_ids
        
        if self.class_data_root: # if there are class images
            class_image = Image.open(self.class_images_path[index % self.num_class_images]) # same idea as above
            if not class_image.mode == "RGB":
                class_image = class_image.convert("RGB")
            example["class_images"] = self.transforms(class_image)
            example["class_prompt_ids"] = self.tokenizer(
                self.class_prompt,
                truncation=True,
                padding="max_length",
                max_length=self.tokenizer.model_max_length,
                return_tensors="pt",
            ).input_ids
            
        return example



In [ ]:
# The Stable Diffusion checkpoint we'll fine-tune
# model_id = "CompVis/stable-diffusion-v1-4"
# model_id = "results1.5/test"
model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"

In [ ]:
# train_dataset = DreamBoothDataset(train, tokenizer)
# train_dataset[0]

In [ ]:
import torch

def collate_fn(examples, with_prior_preservation=False):
    input_ids = [example["instance_prompt_ids"] for example in examples]
    pixel_values = [example["instance_images"] for example in examples]
    
    if with_prior_preservation:
        input_ids += [example["class_prompt_ids"] for example in examples]
        pixel_values += [example["class_images"] for example in examples]
    
    # Stack pixel values
    pixel_values = torch.stack(pixel_values)
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    
    # Stack input ids
    input_ids = torch.stack(input_ids)  # Changed from torch.cat to torch.stack
    
    batch = {
        "input_ids": input_ids,
        "pixel_values": pixel_values,
    }
    return batch

In [ ]:
class PromptDataset(Dataset):
    """A simple dataset to prepare the prompts to generate class images on multiple GPUs.
    The get item returns example dictionary with the prompt and the index of the example.
    """

    def __init__(self, prompt, num_samples):
        """init with promt and number of samples

        Args:
            prompt (str): text prompt
            num_samples (int): num of samples
        """
        self.prompt = prompt
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, index):
        example = {}
        example["prompt"] = self.prompt
        example["index"] = index
        return example


In [ ]:
from argparse import Namespace

args = Namespace(
    pretrained_model_name_or_path=model_id,
    # revision = 'fp16',
    #data
    resolution=512, # Reduce this if you want to save some memory
    seed=1337,
    with_prior_preservation=False,
    class_data_dir = None,#"synthetic_mammograms",
    class_prompt = "a mammogram",
    num_class_images = 100,
    prior_loss_weight = 1.0,
    prior_generation_precision = 'fp16',
    # revision='fp16',
    # instance_prompt=instance_prompt,
    
    #Training
    learning_rate=1e-6,
    rank = 4,
    train_text_encoder=True,
    max_train_steps=1500,
    train_batch_size=4,
    gradient_accumulation_steps=1, # Increase this if you want to lower memory usage
    max_grad_norm=1.0,
    gradient_checkpointing=True,  # Set this to True to lower the memory usage
    use_8bit_adam=True,  # Use 8bit optimizer from bitsandbytes
    dataloader_num_workers = 8,
    lr_scheduler = 'constant',
    lr_warmup_steps = 0,
    lr_num_cycles = 1, #Number of hard resets of the lr in cosine_with_restarts scheduler
    lr_power = 1.0, #Power of the polynomial scheduler
    
     #memory
    enable_xformers_memory_efficient_attention=True,
    allow_tf32 = True, # on Ampere GPU only
    mixed_precision = 'fp16', # fp16 or bf16
    
    #Optimizer
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_weight_decay = 1.0e-2,
    adam_epsilon = 1.0e-8,
    sample_batch_size=4,
    output_dir="results_vini/test",  # Where to save the pipeline
    
    #logging
    validation_prompt = "a mammogram", # validation prompt for logging
    num_validation_images = 4,
    validation_steps = 750, # how often to log validation images
)

In [ ]:
# from diffusers import AutoencoderKL, UNet2DConditionModel
# from transformers import CLIPFeatureExtractor, CLIPTextModel

# text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder")
# vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae")
# unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet")
# # feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
import math

import torch.nn.functional as F
from accelerate import Accelerator
from accelerate.logging import get_logger
import xformers
from accelerate.utils import set_seed
from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    DiffusionPipeline,
    DPMSolverMultistepScheduler,
    UNet2DConditionModel,
    DDIMScheduler,
    StableDiffusionPipeline
)
from transformers import AutoTokenizer, PretrainedConfig, CLIPTextModel
from diffusers.optimization import get_scheduler
from diffusers.utils import check_min_version, is_wandb_available
from diffusers.utils.import_utils import is_xformers_available
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from packaging import version

import wandb

logger = get_logger(__name__)

In [ ]:
def import_model_class_from_model_name_or_path(pretrained_model_name_or_path: str):
    text_encoder_config = PretrainedConfig.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="text_encoder",
    )
    model_class = text_encoder_config.architectures[0]

    if model_class == "CLIPTextModel":
        from transformers import CLIPTextModel

        return CLIPTextModel
    elif model_class == "RobertaSeriesModelWithTransformation":
        from diffusers.pipelines.alt_diffusion.modeling_roberta_series import RobertaSeriesModelWithTransformation

        return RobertaSeriesModelWithTransformation
    elif model_class == "T5EncoderModel":
        from transformers import T5EncoderModel

        return T5EncoderModel
    else:
        raise ValueError(f"{model_class} is not supported.")

In [ ]:
def log_validation(text_encoder, tokenizer, unet, vae, args, accelerator, weight_dtype, epoch):
    """
    Runs validation and logs images to tensorboard and wandb.

    Args:
        text_encoder: The text encoder to use.
        tokenizer: The tokenizer to use.
        unet: The UNet model to use.
        vae: The VAE model to use.
        args: The arguments to use.
        accelerator: The accelerator to use.
        weight_dtype: The data type to use for the weights.
        epoch: The current epoch.
    """
    logger.info(
        f"Running validation... \n Generating {args.num_validation_images} images with prompt:"
        f" {args.validation_prompt}."
    )
    # create pipeline (note: unet and vae are loaded again in float32)
    pipeline = DiffusionPipeline.from_pretrained(
        args.pretrained_model_name_or_path,
        text_encoder=accelerator.unwrap_model(text_encoder),
        tokenizer=tokenizer,
        unet=accelerator.unwrap_model(unet),
        vae=vae,
        # revision=args.revision,
        # torch_dtype=weight_dtype,
        safety_checker=None,
    )
    pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
    pipeline = pipeline.to(accelerator.device)
    pipeline.set_progress_bar_config(disable=True)

    # run inference
    generator = None if args.seed is None else torch.Generator(device=accelerator.device).manual_seed(args.seed)
    images = []
    for _ in range(args.num_validation_images):
        with torch.autocast("cuda"):
            image = pipeline(args.validation_prompt, num_inference_steps=25, generator=generator).images[0]
        images.append(image)
        
    for tracker in accelerator.trackers:
        tracker.log(
            {
                "validation": [
                    wandb.Image(image, caption=f"{i}: {args.validation_prompt}") for i, image in enumerate(images)
                ]
            }
        )

    del pipeline
    torch.cuda.empty_cache()


In [ ]:
def training_function():

    accelerator = Accelerator(
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        mixed_precision=args.mixed_precision, 
        log_with='wandb'
    )
     # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state, main_process_only=False)
    
    set_seed(args.seed)
    
    # Generate class images if prior preservation is enabled.
    if args.with_prior_preservation:
        class_images_dir = Path(args.class_data_dir) # get path of class images
        if not class_images_dir.exists(): # create folder if not exists
            class_images_dir.mkdir(parents=True)
        cur_class_images = len(list(class_images_dir.iterdir())) # count how many images there are

        if cur_class_images < args.num_class_images: # if there are less images than needed
            # define dtype for prior generation
            torch_dtype = torch.float16 if accelerator.device.type == "cuda" else torch.float32
            if args.prior_generation_precision == "fp32":
                torch_dtype = torch.float32
            elif args.prior_generation_precision == "fp16":
                torch_dtype = torch.float16
            elif args.prior_generation_precision == "bf16":
                torch_dtype = torch.bfloat16
            pipeline = DiffusionPipeline.from_pretrained(
                args.pretrained_model_name_or_path, # from same pretrained model as main model
                torch_dtype=torch_dtype,
                safety_checker=None, # no safety checker
            )
            pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
            pipeline.set_progress_bar_config(disable=True)

            num_new_images = args.num_class_images - cur_class_images # number of images still needed
            logger.info(f"Number of class images to sample: {num_new_images}.")

            sample_dataset = PromptDataset(args.class_prompt, num_new_images) # define prompt dataset, returns prompt and index as item
            sample_dataloader = torch.utils.data.DataLoader(sample_dataset, batch_size=args.sample_batch_size)

            sample_dataloader = accelerator.prepare(sample_dataloader) # dataloader for accelerator
            pipeline.to(accelerator.device) # send pipeline to accelerator device

            for example in tqdm(sample_dataloader, desc="Generating class images", disable=not accelerator.is_local_main_process):
                images = pipeline(example["prompt"], num_inference_steps=25).images # generate image(s) from prompt

                for i, image in enumerate(images): # for each image in the batch
                    hash_image = hashlib.sha1(image.tobytes()).hexdigest() # create hash as unique identifier
                    image_filename = class_images_dir / f"{example['index'][i] + cur_class_images}-{hash_image}.jpg"
                    image.save(image_filename)

            del pipeline # free memory when done
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    ##END OF CLASS PRIOR PRESERVATION
    tokenizer = AutoTokenizer.from_pretrained(
        args.pretrained_model_name_or_path,
        subfolder="tokenizer",
        # revision=args.revision,
        use_fast=False
    )
    # import correct text encoder class: only ROBERTA or CLIP
    text_encoder_cls = import_model_class_from_model_name_or_path(args.pretrained_model_name_or_path)
    noise_scheduler = DDPMScheduler.from_pretrained(args.pretrained_model_name_or_path, subfolder="scheduler")
    text_encoder = text_encoder_cls.from_pretrained(
        args.pretrained_model_name_or_path, subfolder="text_encoder",
        # revision=args.revision
    )

    #vae = AutoencoderKL.from_pretrained(args.pretrained_model_name_or_path, subfolder="vae", revision=args.revision)
    vae=AutoencoderKL.from_pretrained(
                    args.pretrained_model_name_or_path,
                    subfolder="vae",
                    # revision=args.revision
                )
    unet = UNet2DConditionModel.from_pretrained(
        args.pretrained_model_name_or_path, subfolder="unet",
        # revision=args.revision
    )

     # freeze models
    vae.requires_grad_(False)
    if not args.train_text_encoder: # freeze text encoder if not training it
        text_encoder.requires_grad_(False)
        
    # For mixed precision training we cast the text_encoder and vae weights to half-precision
    # as these models are only used for inference, keeping weights in full precision is not required.
    weight_dtype = torch.float32
    if accelerator.mixed_precision == "fp16":
        weight_dtype = torch.float16
    elif accelerator.mixed_precision == "bf16":
        weight_dtype = torch.bfloat16
        

     # Move vae and text_encoder to device and cast to weight_dtype
    vae.to(accelerator.device, dtype=weight_dtype)
    if not args.train_text_encoder:
        text_encoder.to(accelerator.device, dtype=weight_dtype)

    # optimize GPU memory usage and more
    if args.enable_xformers_memory_efficient_attention:
        if is_xformers_available():
            import xformers

            xformers_version = version.parse(xformers.__version__)
            if xformers_version == version.parse("0.0.16"):
                logger.warn(
                    "xFormers 0.0.16 cannot be used for training in some GPUs. If you observe problems during training, please update xFormers to at least 0.0.17. See https://huggingface.co/docs/diffusers/main/en/optimization/xformers for more details."
                )
            unet.enable_xformers_memory_efficient_attention()
            vae.enable_xformers_memory_efficient_attention()
        else:
            raise ValueError("xformers is not available. Make sure it is installed correctly")
    
    if args.gradient_checkpointing:
        unet.enable_gradient_checkpointing()
        if args.train_text_encoder:
            text_encoder.gradient_checkpointing_enable()
            
    # # Check that all trainable models are in full precision
    # low_precision_error_string = (
    #     "Please make sure to always have all model weights in full float32 precision when starting training - even if"
    #     " doing mixed precision training. copy of the weights should still be float32."
    # )
    # if accelerator.unwrap_model(unet).dtype != torch.float32:
    #     raise ValueError(
    #         f"Unet loaded as datatype {accelerator.unwrap_model(unet).dtype}. {low_precision_error_string}"
    #     )
    # if args.train_text_encoder and accelerator.unwrap_model(text_encoder).dtype != torch.float32:
    #     raise ValueError(
    #         f"Text encoder loaded as datatype {accelerator.unwrap_model(text_encoder).dtype}."
    #         f" {low_precision_error_string}"
    #     )
        
    
    # Enable TF32 for faster training on Ampere GPUs,
    # cf https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices
    if args.allow_tf32:
        torch.backends.cuda.matmul.allow_tf32 = True
    
    # Use 8-bit Adam for lower memory usage or to fine-tune the model in 16GB GPUs
    if args.use_8bit_adam:
        try:
            import bitsandbytes as bnb
        except ImportError:
            raise ImportError(
                "To use 8-bit Adam, please install the bitsandbytes library: `pip install bitsandbytes`."
            )

        optimizer_class = bnb.optim.AdamW8bit
    else:
        optimizer_class = torch.optim.AdamW

    params_to_optimize = ( # unet and text encoder or une only
        itertools.chain(unet.parameters(), text_encoder.parameters()) if args.train_text_encoder else unet.parameters()
    )
    optimizer = optimizer_class(
        params_to_optimize,
        lr=args.learning_rate,
        betas=(args.adam_beta1, args.adam_beta2),
        weight_decay=args.adam_weight_decay,
        eps=args.adam_epsilon,
    )
    
    train_dataset = DreamBoothDataset(
        train, 
        tokenizer,
        class_data_root=args.class_data_dir if args.with_prior_preservation else None, # prior images
        class_prompt=args.class_prompt, # class (prior) promt
        class_num=args.num_class_images,
    )

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=args.train_batch_size,
        shuffle=True,
        collate_fn=lambda examples: collate_fn(examples, args.with_prior_preservation),
        num_workers = 8
    )
    
    # Scheduler and math around the number of training steps.
    overrode_max_train_steps = False
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
    if args.max_train_steps is None: # if not maximum number of steps is given
        args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
        overrode_max_train_steps = True

    lr_scheduler = get_scheduler(
        args.lr_scheduler,
        optimizer=optimizer,
        num_warmup_steps=args.lr_warmup_steps * args.gradient_accumulation_steps, # if acc steps are present, we need more warm up
        num_training_steps=args.max_train_steps * args.gradient_accumulation_steps, # if more warm up is given, we need more training steps too.
        num_cycles=args.lr_num_cycles,
        power=args.lr_power,
    )


    # Prepare everything with our `accelerator`.
    if args.train_text_encoder:
        unet, text_encoder, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
            unet, text_encoder, optimizer, train_dataloader, lr_scheduler
        )
    else:
        unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
            unet, optimizer, train_dataloader, lr_scheduler
        )
        
    # We need to recalculate our total training steps as the size of the training dataloader may have changed
    num_update_steps_per_epoch = math.ceil(
        len(train_dataloader) / args.gradient_accumulation_steps
    )
    num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)
    
    # We need to initialize the trackers we use, and also store our configuration.
    # The trackers initializes automatically on the main process.
    if accelerator.is_main_process:
        run = "deambooth_mammo_emory_1"
        accelerator.init_trackers(run, config=vars(args)) # add args to wandb

    # Train!
    total_batch_size = (
        args.train_batch_size
        * accelerator.num_processes
        * args.gradient_accumulation_steps
    )
    
    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num batches each epoch = {len(train_dataloader)}")
    logger.info(f"  Instantaneous batch size per device = {args.train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {args.max_train_steps}")
    first_epoch = 0
    
    # Only show the progress bar once on each machine
    progress_bar = tqdm(
        range(args.max_train_steps), disable=not accelerator.is_local_main_process
    )
    progress_bar.set_description("Steps")
    global_step = 0

    for epoch in range(num_train_epochs):
        unet.train()
        if args.train_text_encoder:
            text_encoder.train()
        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(unet):
                # Convert images to latent space
                with torch.no_grad():
                    latents = vae.encode(batch["pixel_values"].to(dtype=weight_dtype)).latent_dist.sample()
                    latents = latents * vae.config.scaling_factor

                # Sample noise that we'll add to the latents
                noise = torch.randn(latents.shape).to(latents.device)
                bsz = latents.shape[0]
                # Sample a random timestep for each image
                timesteps = torch.randint(
                    0,
                    noise_scheduler.config.num_train_timesteps,
                    (bsz,),
                    device=latents.device,
                ).long()

                # Add noise to the latents according to the noise magnitude at each timestep
                # (this is the forward diffusion process)
                noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                # Get the text embedding for conditioning
                encoder_hidden_states = text_encoder(batch["input_ids"])[0]

                 # Predict the noise residual
                model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

                # Get the target for loss depending on the prediction type
                if noise_scheduler.config.prediction_type == "epsilon":
                    target = noise
                elif noise_scheduler.config.prediction_type == "v_prediction":
                    target = noise_scheduler.get_velocity(latents, noise, timesteps)
                else:
                    raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")
                    
                if args.with_prior_preservation:
                    # Chunk the noise and model_pred into two parts and compute the loss on each part separately.
                    try:
                        model_pred, model_pred_prior = torch.chunk(model_pred, 2, dim=0)
                        target, target_prior = torch.chunk(target, 2, dim=0)

                        # Compute instance loss
                        loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

                        # Compute prior loss
                        prior_loss = F.mse_loss(model_pred_prior.float(), target_prior.float(), reduction="mean")

                        # Add the prior loss to the instance loss.
                        loss = loss + args.prior_loss_weight * prior_loss
                    except Exception as e:
                        loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
                else:
                    loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")


                accelerator.backward(loss)
                if accelerator.sync_gradients:
                    params_to_clip = (
                        itertools.chain(unet.parameters(), text_encoder.parameters())
                        if args.train_text_encoder
                        else unet.parameters()
                    )
                    accelerator.clip_grad_norm_(params_to_clip, args.max_grad_norm)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            # Checks if the accelerator has performed an optimization step behind the scenes
            if accelerator.sync_gradients:
                progress_bar.update(1)
                global_step += 1
                
                if accelerator.is_main_process:
                    if args.validation_prompt is not None and global_step % args.validation_steps == 0: # log images to check progress
                        log_validation(text_encoder, tokenizer, unet, vae, args, accelerator, weight_dtype, epoch)

            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)

            if global_step >= args.max_train_steps:
                break

        accelerator.wait_for_everyone()

    # Create the pipeline using the trained modules and save it
    if accelerator.is_main_process:
        print(f"Loading pipeline and saving to {args.output_dir}...")
        pipeline = DiffusionPipeline.from_pretrained(
            args.pretrained_model_name_or_path,
            unet=accelerator.unwrap_model(unet),
            text_encoder=accelerator.unwrap_model(text_encoder),
            vae=AutoencoderKL.from_pretrained(args.pretrained_model_name_or_path, subfolder="vae"),
            safety_checker=None,
        )
        pipeline.save_pretrained(args.output_dir)
        accelerator.end_training()

In [ ]:
from accelerate import notebook_launcher

num_of_gpus = 1  # CHANGE THIS TO MATCH THE NUMBER OF GPUS YOU HAVE
notebook_launcher(
    training_function, num_processes=num_of_gpus
)

In [ ]:
# pipe = StableDiffusionPipeline.from_pretrained(
#     args.output_dir,
#     torch_dtype=torch.float16,
# ).to("cuda")

pipe = StableDiffusionPipeline.from_pretrained(args.output_dir, safety_checker=None, torch_dtype=torch.float16).to("cuda")
generator = torch.Generator(device='cuda')
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
def show_row(imgs, figsize=(10, 10), title="", **kwargs):
    """Show generated images in a row

    Args:
        imgs (list): images list (PILs or numpy arrays)
        figsize (tuple, optional): figure size. Defaults to (10, 10).
        title (str, optional): title. Defaults to "".
    """
    fig, axs = plt.subplots(1, len(imgs), figsize=figsize)
    fig.suptitle(title)
    for i, img in enumerate(imgs):
        axs[i].imshow(img, **kwargs)
        axs[i].axis("off")
    fig.tight_layout()

In [ ]:
idx = random.sample(range(test.shape[0]), 2)
prompt = test['prompt'].values[idx]
prompt

In [ ]:
# Pick a funny prompt here and it will be used as the widget's default 
# when we push to the Hub in the next section

prompts = list(test['prompt'].values[idx])
image_path = test['image_paths'].values[idx][0]

# Tune the guidance to control how closely the generations follow the prompt
# Values between 7-11 usually work best
guidance_scale = [6, 7, 8, 9, 10, 11]

for i in guidance_scale:    
    for prompt in prompts:
        # Seed
        for seed in [1338]:
            generator.manual_seed(seed)
            negative_prompt = ""
            num_samples = 2
            num_inference_steps = 30
            size = 512
            with torch.autocast("cuda"), torch.inference_mode():
                image = pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    num_images_per_prompt=num_samples,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=i,
                    height=size,
                    width=size,
                    generator=generator,
                ).images
            # images in list
            imgs = [np.asarray(img) for img in image]

            show_row(imgs, title=f'Guidance: {i}, prompt: {prompt}', figsize=(20,5), cmap="gray") if len(imgs) > 1 else plt.imshow(imgs[0], cmap="gray")
